In [1]:
import pandas as pd
import time
import tweepy
import csv
import os
from requests.exceptions import Timeout, ConnectionError
import tweepy
import pandas as pd
from tqdm import tqdm

In [167]:
# get twitter_ids for politicians

# read data
df_politician = pd.read_csv("./politician_2016_2022.csv")
user_twitters = df_politician.twitter_handle

user_ids = []
user_jsons = []
for user_screename in tqdm(user_twitters):
    try:
        user_json_list = api.lookup_users(screen_name=[user_screename])
        user_ids.append(user_json_list[0].id_str)
        user_jsons.append(user_json_list[0])
    except Exception as err:
        user_ids.append(None)
        user_jsons.append(None)


100%|██████████| 910/910 [05:07<00:00,  2.96it/s]


In [180]:
# add user ids into `politician_2016_2022.csv`

df_politician['twitter_id'] = user_ids

dem_politician_set = set(df_politician[df_politician.party=="Democratic Party"].twitter_id)
rep_politician_set = set(df_politician[df_politician.party=="Republican Party"].twitter_id)
print(len(dem_politician_set), len(rep_politician_set))

df_politician.to_csv("politician_2016_2022.csv", index=False)

429 438


In [181]:
len(df_politician[df_politician.party=="Democratic Party"].twitter_id)

453

## infer political orientation for covid users

In [182]:
import glob
import os

all_rows = []

for filepath in tqdm(glob.glob("./friends_info/*")):
    user_id = os.path.basename(filepath)[:-4]
    with open(filepath) as f:
        lines = f.read()
        follows = set(lines.split("\t")[1].split(","))
        all_rows.append([user_id, 
                         len(dem_politician_set.intersection(follows)), 
                         len(rep_politician_set.intersection(follows))])

df_poli_follow.to_csv("covid_user_political_follow.csv", index=False)
df_poli_follow.head(5)

100%|██████████| 88124/88124 [09:14<00:00, 159.03it/s]


,twitter_id,dem_count,rep_count
0,108556027,19,2
1,907389979042521089,4,0
2,44372211,0,0
3,710162591486611457,0,0
4,839099917121511425,0,1


In [187]:
df_drug_users = pd.read_csv("./all_users_info.csv")
df_drug_users.user_id = df_drug_users.user_id.astype("str")
df_drug_users.head(3)

,user_id,screen_name,name,description,profile_image_url
0,1274105887926284288,CRoll78,"Kim, WI Nerd in a box of Nerds✝️🇺🇸🇵🇱",Christian w/Jewish ancestry. Conservative Cons...,https://pbs.twimg.com/profile_images/137454645...
1,1217520455013130240,OratorBlog,BLOG-ORATOR,#TheMighty200 An anonymous Patriot who belie...,http://pbs.twimg.com/profile_images/1217548833...
2,268201193,DeccanHerald,Deccan Herald,Bengaluru & Karnataka's leading news daily for...,https://pbs.twimg.com/profile_images/127500460...


In [188]:
df_drug_users = df_drug_users.merge(df_poli_follow, left_on=["user_id"], right_on=["twitter_id"])
df_drug_users.head(3)

,user_id,screen_name,name,description,profile_image_url,twitter_id,dem_count,rep_count
0,1274105887926284288,CRoll78,"Kim, WI Nerd in a box of Nerds✝️🇺🇸🇵🇱",Christian w/Jewish ancestry. Conservative Cons...,https://pbs.twimg.com/profile_images/137454645...,1274105887926284288,0,2
1,268201193,DeccanHerald,Deccan Herald,Bengaluru & Karnataka's leading news daily for...,https://pbs.twimg.com/profile_images/127500460...,268201193,0,0
2,1347730009084059653,philipM21152135,Philip Moses,"Father, Husband,Mental Health Community Social...",https://pbs.twimg.com/profile_images/134773179...,1347730009084059653,29,0


In [189]:
def assign_party(row):
    if row['dem_count'] == row['rep_count'] == 0:
        return "neutral"
    elif row['dem_count'] <= row['rep_count']:
        return "left"
    else:
        return "right"

df_drug_users['political_orientation'] = df_drug_users.apply(assign_party, axis=1)
df_drug_users.head(3)

,user_id,screen_name,name,description,profile_image_url,twitter_id,dem_count,rep_count,political_orientation
0,1274105887926284288,CRoll78,"Kim, WI Nerd in a box of Nerds✝️🇺🇸🇵🇱",Christian w/Jewish ancestry. Conservative Cons...,https://pbs.twimg.com/profile_images/137454645...,1274105887926284288,0,2,left
1,268201193,DeccanHerald,Deccan Herald,Bengaluru & Karnataka's leading news daily for...,https://pbs.twimg.com/profile_images/127500460...,268201193,0,0,neutral
2,1347730009084059653,philipM21152135,Philip Moses,"Father, Husband,Mental Health Community Social...",https://pbs.twimg.com/profile_images/134773179...,1347730009084059653,29,0,right


In [190]:
df_drug_users[df_drug_users.political_orientation == "left"].shape

(17154, 9)

In [191]:
df_drug_users[df_drug_users.political_orientation == "right"].shape

(34989, 9)

In [192]:
df_drug_users[df_drug_users.political_orientation == "neutral"].shape

(35981, 9)